In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/wrong_data.csv')

In [ ]:
dataset.iloc[10, 2]

In [ ]:
!pip install transformers
!pip install accelerate
!pip install sentencepiece

In [ ]:
! pip install happytransformer

In [ ]:
import json

In [ ]:
json_data = []

In [ ]:
for i, j in list(zip(list(dataset['wrong']), list(dataset['correct']))):
  json_data.append({'input_text' : i, 'output_text' : j})

In [ ]:
json.dump(json_data, open('train.json', 'w'))

In [ ]:
! pip install datasets

In [ ]:
!pip install sentencepiece

In [ ]:
data = json.load(open('/content/drive/MyDrive/train.json'))[:20000]

In [ ]:
data_l = json.load(open('/content/drive/MyDrive/train_new.json'))

In [ ]:
import random

In [ ]:
to_extend = random.sample(data_l, 100000)

In [ ]:
import tensorflow as tf

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets
import json

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length, max_output_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        input_text = item['input_text']
        output_text = item['output_text']

        # Tokenize and truncate input and output
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_input_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_attention_mask=True,
        )

        output_encoding = self.tokenizer(
            output_text,
            max_length=self.max_output_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": input_encoding.input_ids.flatten(),
            "attention_mask": input_encoding.attention_mask.flatten(),
            "labels": output_encoding.input_ids.flatten(),
        }

# Load your data
data_l.extend(json.load(open('/content/drive/MyDrive/new_data.json')))



# Optional: Evaluate the model if you have an evaluation dataset
# evaluation_dataset = ...
# trainer.evaluate(eval_dataset=evaluation_dataset)


In [ ]:
train_data = CustomDataset(random.sample(data_l, 100000), tokenizer, max_input_length, max_output_length)

In [ ]:
test_data = CustomDataset(data, tokenizer, max_input_length, max_output_length)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = T5ForConditionalGeneration.from_pretrained("google/mt5-base")

In [ ]:
max_input_length = 128  # You can adjust this based on your dataset
max_output_length = 128  # You can adjust this based on your dataset
#custom_dataset = CustomDataset(data[:400], tokenizer, max_input_length, max_output_length)

In [ ]:
import random
train_data = random.sample(data, 7512)
test_data = list(filter(lambda x : x not in train_data, data))

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/wiki_extracts.csv')

In [ ]:
eval_data = []
for i, j in list(zip(list(df['input']), list(df['target']))):
  eval_data.append({'input_text' : i, 'output_text' : j})

In [ ]:
import random
train_data = random.sample(eval_data, 12000)
test_data = list(filter(lambda x : x not in train_data, eval_data))

In [ ]:
train_data = CustomDataset(train_data, tokenizer, max_input_length, max_output_length)
test_data = CustomDataset(test_data, tokenizer, max_input_length, max_output_length)

In [ ]:
len(train_data), len(test_data)

In [ ]:
import pickle
with open('train_data.pkl', 'rb') as f:
  train_data = pickle.load(f)
with open('test_data.pkl', 'rb') as f:
  test_data = pickle.load(f)

In [ ]:
import pickle
with open("train_data.pkl" , "wb") as f:
  pickle.dump(train_data, f)
with open("test_data.pkl" , "wb") as f:
  pickle.dump(test_data, f)

In [ ]:
eval_dataset = CustomDataset(eval_data, tokenizer, max_input_length, max_output_length)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    save_steps=500,
)

# Initialize Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data
)
trainer.train()

In [ ]:
!zip -r results_v3.zip ./results/model ./results/tokenizer

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./results/model")

# You can also save the tokenizer
tokenizer.save_pretrained("./results/tokenizer")

In [ ]:
!zip -r results.zip

In [ ]:
!cp results_v5.zip /content/drive/MyDrive/results_v5.zip

In [ ]:
import shutil
shutil.rmtree("./results/model")
shutil.rmtree("./results/tokenizer")

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./results_v5/results/model/")
tokenizer = AutoTokenizer.from_pretrained("./results_v5/results/tokenizer/")

In [ ]:
import random

In [ ]:
ch = random.choice(data)
inputs = tokenizer("", return_tensors="pt")
max_length = 128  # Adjust as needed
generated = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
corrected_text = tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
print("Prediction\t" + corrected_text)
print("Expected\t" + ch['output_text'])
print("Wrong text\t" + ch['input_text'])
print("Corrected ? : " + corrected_text == ch['output_text'])

In [ ]:
! cp /content/drive/MyDrive/results_v5.zip .

In [ ]:
import zipfile
with zipfile.ZipFile("results_v5.zip", 'r') as zip_ref:
    zip_ref.extractall("./results_v5")